# The capital asset pricing model and the security market line

In [1]:
""" 
Linear regression with SciPy 
"""
from scipy import stats

stock_returns = [0.065, 0.0265, -0.0593, -0.001, 0.0346]
mkt_returns = [0.055, -0.09, -0.041, 0.045, 0.022]
beta, alpha, r_value, p_value, std_err = \
    stats.linregress(stock_returns, mkt_returns) # 회귀선의 기울기, 회귀선의 절편, 상관계수, 기울기0 귀물가설하의 p-값, 추정 표준오차

In [8]:
print(beta, alpha) # 기울기와 절편

0.5077431878770808 -0.008481900352462384


무위험 이자율 5%, 시장 위험 프리미엄 8.5%  
0.5077 * 8.5 = 4.3  
4.3 + 5 = 9.3 - 예상 자본 수익률

### APT = Arbitrage Pricing Theory  
여러 체계적 위험 요소의 선형 조합으로 구성된   
다중 요소 모델에 의해 증권의 수익이 생성된다  
(인플레이션율, GDP 성장률, 실질 이자율, 배당금)

# Multivariate linear regression of factor models

In [3]:
# pip install -U statsmodels

Note: you may need to restart the kernel to use updated packages.


In [4]:
""" 
Least squares regression with statsmodels 
"""
import numpy as np
import statsmodels.api as sm

# Generate some sample data
num_periods = 9 # 9개의 기간 동안 측정
all_values = np.array([np.random.random(8) \
                       for i in range(num_periods)])

# Filter the data
y_values = all_values[:, 0] # First column values as Y
x_values = all_values[:, 1:] # All other values as X
x_values = sm.add_constant(x_values) # Include the intercept
results = sm.OLS(y_values, x_values).fit() # Regress and fit the model

In [6]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.948
Model:                            OLS   Adj. R-squared:                  0.580
Method:                 Least Squares   F-statistic:                     2.578
Date:                Thu, 23 May 2024   Prob (F-statistic):              0.447
Time:                        11:34:32   Log-Likelihood:                 9.7628
No. Observations:                   9   AIC:                            -3.526
Df Residuals:                       1   BIC:                            -1.948
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6533      0.557      1.174      0.4

c:\Users\wooyong\anaconda3\envs\finance\lib\site-packages\scipy\stats\stats.py:1542: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  "anyway, n=%i" % int(n))


In [7]:
print(results.params) # a x1~x7까지의 값들 제공

[ 0.65330956  0.12416589  0.95917575 -0.72659921 -0.13383018 -0.71741458
 -1.00989633  0.43011211]


# Linear optimization

In [8]:
# pip install pulp # open source 선형 프로그래밍 모델

     ---------------------------------------- 17.7/17.7 MB 2.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


## A maximization example with linear programming

In [9]:
""" 
A linear optimization problem with 2 variables
"""
import pulp

x = pulp.LpVariable('x', lowBound=0) # 함수를 풀어야 할 변수 설정
y = pulp.LpVariable('y', lowBound=0) # 

problem = pulp.LpProblem(
    'A simple maximization objective', 
    pulp.LpMaximize)
problem += 3*x + 2*y, 'The objective function' # += 텍스트 설명과 임의 개수의 제약조건 추가 가능
problem += 2*x + y <= 100, '1st constraint'
problem += x + y <= 80, '2nd constraint'
problem += x <= 40, '3rd constraint'
problem.solve()

c:\Users\wooyong\anaconda3\envs\finance\lib\site-packages\pulp\pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


1

In [16]:
print("Maximization Results:")
for variable in problem.variables():
    print(variable.name, '=', variable.varValue)

Maximization Results:
x = 20.0
y = 60.0


## A minimization example with integer programming

In [10]:
""" 
An example of implementing an integer 
programming model with binary conditions 
"""
import pulp

dealers = ['X', 'Y', 'Z']
variable_costs = {'X': 500, 'Y': 350, 'Z': 450}
fixed_costs = {'X': 4000, 'Y': 2000, 'Z': 6000}

# Define PuLP variables to solve
quantities = pulp.LpVariable.dicts('quantity', 
                                   dealers, 
                                   lowBound=0,
                                   cat=pulp.LpInteger)
is_orders = pulp.LpVariable.dicts('orders', 
                                  dealers,
                                  cat=pulp.LpBinary)

In [ ]:
"""
This is an example of implementing an integer programming model
with binary variables the wrong way.
"""
# Initialize the model with constraints
model = pulp.LpProblem('A cost minimization problem',
                       pulp.LpMinimize)
model += sum([(variable_costs[i] * \
               quantities[i] + \
               fixed_costs[i])*is_orders[i] \
              for i in dealers]), 'Minimize portfolio cost'
model += sum([quantities[i] for i in dealers]) == 150\
    , 'Total contracts required'
model += 30 <= quantities['X'] <= 100\
    , 'Boundary of total volume of X'
model += 30 <= quantities['Y'] <= 90\
    , 'Boundary of total volume of Y'
model += 30 <= quantities['Z'] <= 70\
    , 'Boundary of total volume of Z'
model.solve()  # You will get an error running this code!  + 비선형 프로그래밍으 수행하게 됨

In [13]:
"""
This is an example of implementing an 
IP model with binary variables the correct way.
"""
# Initialize the model with constraints
model = pulp.LpProblem('A cost minimization problem',
                       pulp.LpMinimize)
model += sum(
    [variable_costs[i]*quantities[i] + \
         fixed_costs[i]*is_orders[i] for i in dealers])\
    , 'Minimize portfolio cost'
model += sum([quantities[i] for i in dealers]) == 150\
    ,  'Total contracts required'
model += is_orders['X']*30 <= quantities['X'] <= \
    is_orders['X']*100, 'Boundary of total volume of X' # IS_ORDERS 미지수, 모든 미지수를 선형 방식으로 배치해야 함
model += is_orders['Y']*30 <= quantities['Y'] <= \
    is_orders['Y']*90, 'Boundary of total volume of Y'
model += is_orders['Z']*30 <= quantities['Z'] <= \
    is_orders['Z']*70, 'Boundary of total volume of Z'
model.solve()

c:\Users\wooyong\anaconda3\envs\finance\lib\site-packages\pulp\pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


1

In [14]:
print('Minimization Results:')
for variable in model.variables():
    print(variable, '=', variable.varValue)

print('Total cost:',  pulp.value(model.objective))

Minimization Results:
orders_X = 0.0
orders_Y = 1.0
orders_Z = 1.0
quantity_X = 0.0
quantity_Y = 90.0
quantity_Z = 60.0
Total cost: 66500.0


# Solving linear equations using matrices

In [43]:
""" 
Linear algebra with NumPy matrices 
"""
import numpy as np

A = np.array([[2, 1, 1],[1, 3, 2],[1, 0, 0]])
B = np.array([4, 5, 6])

Use the `linalg.solve` function of NumPy to solve a system of linear scalar
equations:

In [44]:
print(np.linalg.solve(A, B))

[  6.  15. -23.]


# The LU decomposition

### LU decomposition - 하단-상단 분해(lower-upper factorization) = 선형 정방 연립방정식을 푸는 방법 중 하나

In [15]:
 """ 
LU decomposition with SciPy 
"""
import numpy as np
import scipy.linalg as linalg


# Define A and B
A = np.array([
    [2., 1., 1.],
    [1., 3., 2.],
    [1., 0., 0.]])
B = np.array([4., 5., 6.])

# Perform LU decompositiopn
LU = linalg.lu_factor(A) # LU 변수를 A 행렬의 피벗된 LU 분해를 제공함
x = linalg.lu_solve(LU, B)

In [16]:
print(x)

[  6.  15. -23.]


In [17]:
import scipy

P, L, U = scipy.linalg.lu(A)

print('P=\n', P) # 순열 행렬
print('L=\n', L) # 하삼각 행렬
print('U=\n', U) # 상삼각 행렬

P=
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
L=
 [[ 1.   0.   0. ]
 [ 0.5  1.   0. ]
 [ 0.5 -0.2  1. ]]
U=
 [[ 2.   1.   1. ]
 [ 0.   2.5  1.5]
 [ 0.   0.  -0.2]]


# The Cholesky decomposition

### 대칭 행렬의 속성을 사용  
### LU 분해보다 훨씬 더 빠르고 메모리를 훨씬 적게 사용함
### Hermitian(또는 실수값 대칭이므로 정방행렬) 양의 정부호여야 함


In [28]:
""" 
Cholesky decomposition with NumPy 
"""
import numpy as np

A = np.array([
    [10., -1., 2., 0.],
    [-1., 11., -1., 3.],
    [2., -1., 10., -1.],
    [0., 3., -1., 8.]])
B = np.array([6., 25., -11., 15.])

L = np.linalg.cholesky(A)

In [29]:
print(L)

[[ 3.16227766  0.          0.          0.        ]
 [-0.31622777  3.3015148   0.          0.        ]
 [ 0.63245553 -0.24231301  3.08889696  0.        ]
 [ 0.          0.9086738  -0.25245792  2.6665665 ]]


In [30]:
print(np.dot(L, L.T.conj()))  # A=L.L*

[[10. -1.  2.  0.]
 [-1. 11. -1.  3.]
 [ 2. -1. 10. -1.]
 [ 0.  3. -1.  8.]]


In [31]:
y = np.linalg.solve(L, B)  # L.L*.x=B; When L*.x=y, then L.y=B

In [32]:
x = np.linalg.solve(L.T.conj(), y)  # x=L*'.y

In [33]:
print(x)

[ 1.  2. -1.  1.]


In [34]:
print(np.mat(A) * np.mat(x).T)  # B=Ax

[[  6.]
 [ 25.]
 [-11.]
 [ 15.]]


cholesky 한 번 더 공부하기

# The QR decomposition

In [35]:
""" 
QR decomposition with scipy 
"""
import numpy as np
import scipy.linalg as linalg


A = np.array([
    [2., 1., 1.],
    [1., 3., 2.],
    [1., 0., 0]])
B = np.array([4., 5., 6.])

Q, R = scipy.linalg.qr(A)  # QR decomposition
y = np.dot(Q.T, B)  # Let y=Q'.B
x = scipy.linalg.solve(R, y)  # Solve Rx=y

In [36]:
print(x)

[  6.  15. -23.]


# Solving with other matrix algebra methods

## The Jacobi method

### 계산을 병렬적으로 수행, 더 빠른 계산 방법  
### A 행렬이 절대 비축소 대각 우세(strictly irreducibly diagonally dominant)인 경우 수렴이 보장됨  
### 모든 행의 대각선 요소의 절댓값이 다른 항의 절댓값의 합보다 큰 행렬

In [37]:
"""
Solve Ax=B with the Jacobi method 
"""
import numpy as np

def jacobi(A, B, n, tol=1e-10):
    # Initializes x with zeroes with same shape and type as B
    x = np.zeros_like(B)

    for iter_count in range(n):
        x_new = np.zeros_like(x)
        for i in range(A.shape[0]):
            s1 = np.dot(A[i, :i], x[:i])
            s2 = np.dot(A[i, i + 1:], x[i + 1:])
            x_new[i] = (B[i] - s1 - s2) / A[i, i]

        if np.allclose(x, x_new, tol):
            break

        x = x_new

    return x

x1, x2, x3에 대한 값을 반복을 통해 수렴값으로 도달

In [38]:
A = np.array([
    [10., -1., 2., 0.], 
    [-1., 11., -1., 3.], 
    [2., -1., 10., -1.], 
    [0.0, 3., -1., 8.]])
B = np.array([6., 25., -11., 15.])
n = 25

In [39]:
x = jacobi(A, B, n)
print('x', '=', x)

x = [ 1.  2. -1.  1.]


# The Gauss-Seidel method

In [40]:
""" 
Solve Ax=B with the Gauss-Seidel method 
"""
import numpy as np


def gauss(A, B, n, tol=1e-10):
    L = np.tril(A)  # returns the lower triangular matrix of A
    U = A-L  # decompose A = L + U
    L_inv = np.linalg.inv(L)  # 역행렬
    x = np.zeros_like(B)

    for i in range(n):
        Ux = np.dot(U, x)
        x_new = np.dot(L_inv, B - Ux)

        if np.allclose(x, x_new, tol): # 이전 해와 새로운 해의 허용 오차내에 존재하는 지 확인
            break

        x = x_new

    return x

In [41]:
A = np.array([
    [10., -1., 2., 0.], 
    [-1., 11., -1., 3.], 
    [2., -1., 10., -1.], 
    [0.0, 3., -1., 8.]])
B = np.array([6., 25., -11., 15.])
n = 100
x = gauss(A, B, n)

In [42]:
print('x', '=', x)

x = [ 1.  2. -1.  1.]


LU분해, 촐레스키분해, QR분해